In [5]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
from urllib2 import urlopen
import urllib2
from bs4 import BeautifulSoup
import io
import pandas as pd
from io import StringIO, BytesIO
import pandas as pd
import re
import time
from tqdm import tqdm
from multiprocessing import Pool
import os

# Intro
parser of http://www.chrono24.com with roll-out to csv

# Part 1 - items
item parsing

In [23]:
# table raws
class BasicRaw:
    def __init__(self):
        self.text = "None"
    def get_text(self):
        return self.text.encode("utf-8").strip()
class TitleRaw(BasicRaw):
    def __init__(self, title):
        self.text = title

class KeyValueRaw(BasicRaw):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.text = key+" "+value
    def get_kvp(self):
        return self.key, self.value
class FunctionsAndOther(BasicRaw):
    def __init__(self, string, prev_header="None"):
        self.text = string.encode("utf-8")
        self.prev_header = prev_header
class Description(BasicRaw):
    def __init__(self, text):
        self.text = text.encode("utf-8")
    def __str__(self):
        return self.text.encode("utf-8")
class Raw:
    # takes raw raw basically from tbody tag:) and that return one of TitleRow or KeyValueRaw object
    # prev is previous title
    def __init__(self, soup, prev="None"):
        self.soup = soup
        self.prev = prev
    def get_particular_raw_data(self):
        req = self.soup.find('span', id="watchNotes")
        if not(req is None):
            return Description(req.text.encode("utf-8"))
        req = self.soup.find('h4')
        if not(req is None):
            # find a header raw
            return TitleRaw(req.text.encode("utf-8"))
        req = self.soup.find_all('td')
        if len(req) == 2:
            # found key-value pair (we hope)
            return KeyValueRaw(req[0].text.strip(), req[1].text.strip())
        else:
            return FunctionsAndOther(','.join([i.text for i in req]), self.prev)

In [24]:
class BasePage:
    def __init__(self, url, proxy_seed = 0, proxies=None):
        self.url = url + "?SETLANG=en_US&SETCURR=EUR"
        if proxies is None:
            self.proxies = []
        else:
            self.proxies = proxies
        self.ip_usage = proxy_seed
        # get's beautiful soup of page
        self.set_soup()
    def get_page(self):
        # open new ip each time using counter ip_usage
        prx = self.ip_usage % (len(self.proxies) + 1)
        self.ip_usage = (self.ip_usage + 1) % (len(self.proxies) + 1)
        if (prx == 0):
            test_proxy = urllib2.ProxyHandler({})
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            return urllib2.urlopen(self.url)
        else:
            prx -= 1
            test_proxy = urllib2.ProxyHandler(self.proxies[prx])
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            answer = urllib2.urlopen(self.url)
            return answer
    def set_soup(self):
        page = self.get_page()
        soup = BeautifulSoup(page, 'html.parser')
        self.soup = soup

In [25]:
class Page(BasePage):
    def __init__(self, url, proxy_seed = 0, proxies=None):
        """
            this class scrap data form web page that include all info about item
            proxies - array of proxy ips
            proxy_seed is variable that determine proxy_id - can be the counter of Page class usage
        """
        BasePage.__init__(self, url, proxy_seed, proxies)
        # all data form rows
        self.rows = []
    def get_table(self):
        # put all data from tables in array of raws 
        # where each raw is a particular object selected in Raw class
        
        # get part of page where table is

        # self.soup = self.soup.find('div', class_="full-content").find('div', class_="container")
        data = []
        for table in self.soup.find_all('tbody'):
            # this is title object /is a inner rows-sub-box title/
            prev = "None"
            for raw in table.find_all('tr'):
                r = Raw(raw, prev)
                partic_obj = r.get_particular_raw_data()
                if isinstance(partic_obj, TitleRaw):
                    prev = partic_obj
                data.append(partic_obj)
        return data
    def get_js_data(self):
        data = self.soup.find_all('script')
        var_names = ["watchId", "merchantId", "watchName", "originalPath", "uuid", "merchantPage"]
        data_dict = {}
        for var_name in var_names:
            for i in data:
                values = re.findall(r'var {} = "(.*?)";'.format(var_name), str(i), re.DOTALL | re.MULTILINE)
                if len(values) != 0:
                    data_dict[var_name] = values[0]
        return data_dict
    def get_from_table(self):
        # get pandas object containing all data from raws
        raws = self.get_table()
        data = self.get_js_data()
        for raw in raws:
            if isinstance(raw, KeyValueRaw):
                r_kvp = raw.get_kvp()
                if r_kvp[0] == u'\xa0' or r_kvp[0] == u'\u20ac':
                    r_kvp = ["_OTHER_INFO_", r_kvp[1]]
                if r_kvp[0] in data:
                    # if keys collided
                    continue
                data[r_kvp[0]] = r_kvp[1]
            elif isinstance(raw, Description):
                data["_Description_"] = raw.get_text()
            elif isinstance(raw, FunctionsAndOther):
                if "_Functions_and_other" in data:
                    data["_Functions_and_other"] += '; ' + raw.get_text()
                else:
                    data["_Functions_and_other"] = raw.get_text()
        price = self.get_price()
        data["__price__"] = price
        return data
    def get_price(self):
        try:
            req = self.soup.find('span', class_='price-lg')
            out = req.text.strip().encode('utf-8')
        except Exception, e:
            return "None"
        # print req.find('span', class_='currency').text
        return out

## tests

In [26]:
test_item_url = "http://www.chrono24.com/chronoswiss/ch-7523-lunar-chronograph-ss-25186--id6294206.htm"

In [27]:
page = Page(test_item_url)

In [28]:
page.get_table()
page.get_price()

'2,905\xc2\xa0\xe2\x82\xac'

In [29]:
page.get_from_table()

{u'': u'With box',
 u'Availability': u'Available now',
 u'Brand': u'Chronoswiss',
 u'Case diameter': u'38 mm',
 u'Case material': u'Steel',
 u'Clasp': u'Fold clasp',
 u'Clasp material': u'Steel',
 u'Code': u'25186',
 u'Condition': u'1 (mint)',
 u'Dial': u'Silver',
 u'Glass': u'Sapphire Glass',
 u'Location': u'United States, Massachusetts, Boston',
 u'Movement': u'Automatic',
 u'Price': u'$3,200 (= 2,905\xa0\u20ac)',
 u'Ref. No.': u'7523',
 u'Thickness': u'14.5 mm',
 '_Description_': "Chronoswiss CH 7523 Lunar Chronograph, CH 7523 L, CH7523L, stainless steel on a strap with a stainless steel deployant buckle, automatic, date, chronograph, moon phases, small second indicator at 9 o'clock, silver gullioche dial, diameter: 38 mm, thickness: 14.5 mm, display back, sapphire crystal, water resistant, Like New with original box and blank warranty certificate.\n\n\n     $3,200",
 '_Functions_and_other': 'Moonphase, Chronograph, Date; Display Back, Small Seconds',
 '__price__': '2,905\xc2\xa0\xe

In [30]:
page.get_js_data()

{'merchantId': '2092',
 'originalPath': '/chronoswiss/ch-7523-lunar-chronograph-ss-25186--id6294206.htm',
 'uuid': '244475da-8820-452f-bd76-e15f1afeda1d',
 'watchId': '6294206',
 'watchName': 'Chronoswiss CH 7523 Lunar Chronograph SS (25186)'}

In [31]:
pd.DataFrame(page.get_from_table(), index=[0])

,,Availability,Brand,Case diameter,Case material,Clasp,Clasp material,Code,Condition,Dial,...,Ref. No.,Thickness,_Description_,_Functions_and_other,__price__,merchantId,originalPath,uuid,watchId,watchName
0,With box,Available now,Chronoswiss,38 mm,Steel,Fold clasp,Steel,25186,1 (mint),Silver,...,7523,14.5 mm,"Chronoswiss CH 7523 Lunar Chronograph, CH 7523...","Moonphase, Chronograph, Date; Display Back, Sm...","2,905 €",2092,/chronoswiss/ch-7523-lunar-chronograph-ss-2518...,244475da-8820-452f-bd76-e15f1afeda1d,6294206,Chronoswiss CH 7523 Lunar Chronograph SS (25186)


# Part 2 - brands
brands parsing

In [39]:
dir_name = 0
def extract_data(url):
    mainData = dict()
    sz = 0

    to_csv=True
    txt = urllib2.urlopen(url).read()
    data = BeautifulSoup(txt, 'html.parser')
    hrefs = data.find_all("div", class_="item")
    urls = []
    for h in hrefs:
        url_add = h.find("a")["href"]
        urls.append(main_page + url_add)
    for i in tqdm(urls):
        sz += 1
        page_ = Page(i)
        dct = page_.get_from_table()
        for k in dct.keys():
            if k in mainData:
                mainData[k].append(dct[k])
            else:
                mainData[k] = ["None"] * (sz - 1)
                mainData[k].append(dct[k])
        for k in mainData:
            if not k in dct:
                mainData[k].append("None")

    data_to_csv = pd.DataFrame(mainData)
    if to_csv:
        dir_ = page
        NAME = re.findall(r'index*-(.*?).htm', url)
        if len(NAME) == 0:
            NAME = "1"
        else:
            NAME = NAME[0]
        data_to_csv.to_csv("./{}.csv".format(NAME), sep=";")
    

class Pagination:
    def __init__(self, soup):
        # extract last existing page
        self.soup = soup
    def find_max_index(self):
        return int(str(self.soup.find("ul", class_="pagination").find_all('li')[-2].text).strip())
        
class ModelPage(BasePage):
    # class that extract links form list-page
    def __init__(self, page_url="http://www.chrono24.com/chronoswiss", adder="/index.htm", proxy_seed=0, proxies=None):
        # get_page from url acording to the pattern "http://www.chrono24.com/chronoswiss" + /index.htm or /index-{i > 1}.htm adder
        
        # dict that contains all raws of structed data as a dict of {'column_id':[array of raws]}
        
        
        # initialization of base class
        BasePage.__init__(self, page_url, proxy_seed, proxies)
        
        # raws counter
       
        self.extractor = extract_data
        
        self.main_page = page_url
        self.adder = adder
        pagination = Pagination(self.soup)
        # find max index of page list items
        self.max_index = pagination.find_max_index() + 1
        print self.max_index
        self.adders = map(lambda x: self.main_page + x, ["/index.htm"] + ["/index-{}.htm".format(i) for i in range(2, self.max_index)])
        
    def extract_all(self):
        pool = Pool(4)
        pool.map(self.extractor, self.adders)
        pool.close()
        pool.join()
    def calc(self):
        pagination = Pagination(self.soup)
        return self.max_index

In [40]:
main_page = "http://www.chrono24.com/chronoswiss"
adder = "/index.htm?SETLANG=en_US&SETCURR=EUR"
# pages ase ..index, index-2, index-3, index-4

In [41]:
from os.path import dirname, abspath


class MakeFolder:
    def __init__(self, urls, my_path="/Users/istar/Chrono_data"):
        """
        this class start procesess via ModePage
        """
        self.id_ = 0
        self.urls = urls
        self.my_path = my_path
    def start(self):
        os.chdir(self.my_path)
        try:
            for url in self.urls:
                print "Started: " + url
                self.id_ += 1
                NAME = 'Chrono_d2u{}'.format(re.findall(r'http://www.chrono24.com/(.*?)/index.htm', url)[0])
                try:
                    os.mkdir(NAME)
                except Exception, e:
                    print "DIR " + str(e)
                os.chdir("./{}".format(NAME))
                model = ModelPage(url)
                model.extract_all()
                print "Finished: " + url
                os.chdir(self.my_path)
        except Exception, e:
            print "MAKEFOLDER " + str(e)
    def calc(self):
        ans = 0
        for url in self.urls:
            model = ModelPage(url)
            ans += model.calc()
        return ans


In [42]:
MakeFolder(["http://www.chrono24.com/aquastar/index.htm"]).start()

In [43]:
import string
# string.ascii_uppercase
class BrandExtractor(BasePage):
    def __init__(self, page_url="http://www.chrono24.com/search/browse.htm", proxy_seed=0, proxies=None):
        BasePage.__init__(self, page_url, proxy_seed, proxies)
    def get_links(self):
        links = []
        for liter in string.ascii_uppercase:
            links = links + map(lambda x: x.find('a')['href'], self.soup.find('div', id="js-brands-{}".format(liter)).find('ul', class_="list-unstyled").find_all('li'))
        return links

## tests

In [44]:
brands = BrandExtractor()

In [45]:
brands.get_links()

[u'/audemarspiguet/index.htm',
 u'/aerowatch/index.htm',
 u'/aquastar/index.htm',
 u'/alfredrochatfils/index.htm',
 u'/atlantic/index.htm',
 u'/angularmomentum/index.htm',
 u'/apple/index.htm',
 u'/aigner/index.htm',
 u'/anonimo/index.htm',
 u'/alpina/index.htm',
 u'/auricoste/index.htm',
 u'/aquanautic/index.htm',
 u'/arnoldson/index.htm',
 u'/aristo/index.htm',
 u'/altanus/index.htm',
 u'/adidas/index.htm',
 u'/azimuth/index.htm',
 u'/andreasstrehler/index.htm',
 u'/angelus/index.htm',
 u'/armandnicolet/index.htm',
 u'/arminstrom/index.htm',
 u'/azzaro/index.htm',
 u'/alfreddunhill/index.htm',
 u'/alangesoehne/index.htm',
 u'/abpparis/index.htm',
 u'/alainsilberstein/index.htm',
 u'/andersengenve/index.htm',
 u'/augustereymond/index.htm',
 u'/askania/index.htm',
 u'/artya/index.htm',
 u'/armani/index.htm',
 u'/ateliersdemonaco/index.htm',
 u'/alexandershorokhoff/index.htm',
 u'/balmain/index.htm',
 u'/benrus/index.htm',
 u'/bubenzoerweg/index.htm',
 u'/boucheron/index.htm',
 u'/berto

## get started

In [50]:
def main_start(start=0, finish=3, ans=0, ):
    """
        this function gets Folder per brand
        so you can call it with different start parameters
        to start parsing form maybe 100th brand (start=100)
    """
    brands = BrandExtractor()
    try:
        i = start
        for brand in tqdm(brands.get_links()[i:finish]):
            url = "http://www.chrono24.com" + brand
            MakeFolder([url], "/Users/istar/Chrono_data").start()
            print "OK" + brand
            i += 1
    except Exception, e:
        print e
        time.sleep(1)
        main_start(i + 1, ans, finish)

In [ ]:
main_start(3, 5)
# gets brands form 3-rd to 5-th according to brands.get_links()

# Part 3 - collecting 
collect all info in one csv file

In [71]:
class Collector:
    def __init__(self, path="/Users/istar/Chrono_data"):
        # jumping in sub-dirs
        if path is None:
            path = os.curdir
        all_data = []
        for d in os.listdir(path):
            # range(brands)
            if 'Chrono' != d[:6]:
                continue
            os.chdir(path+"/"+d)
            df = []
            try:
                df = self.get_df(path+"/"+d)
                all_data.append(df)
            except Exception, e:
                print e
                os.chdir(path)
            os.chdir(path)
        self.data = pd.concat([_ for _ in all_data])
    def get_df(self, path):
        print path
        allf = os.listdir(path)
        return pd.concat([pd.read_csv(path + "/" + _, sep=";") for _ in allf])

### using example

In [ ]:
path = "/Users/istar/Chrono_data/Chrono_d2ualainsilberstein"
allf = os.listdir(path)

In [ ]:
df = pd.DataFrame()
pd.concat([pd.read_csv(path + "/" + _, sep=";") for _ in allf])

In [ ]:
pd.read_csv(path + "/" + file_name, sep=";")

In [ ]:
pd.concat((df,pd.read_csv(path + "/" + file_name, sep=";"), pd.read_csv(path + "/" + "1.csv", sep=";")), axis=0)

In [ ]:
coll = Collector()

In [ ]:
# treat indexing
df = coll.data.reset_index(drop=True)